In [ ]:
#Import Necessary Libraries
import time # Library for using time delays
from datetime import datetime # Library to get Current Date and Time
import threading # To run multiple process in Parallel
from picamera import PiCamera # To use Pi Camera
import RPi.GPIO as GPIO # For Using GPIO Pins
import sys # System Library
import requests # To ping IFTTT
import adafruit_dht #DHT sensor library


GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)
#Intitializing Moisture sensor with its sensor pin connected to
channel = 21
GPIO.setup(channel, GPIO.IN)


# Initial the dht device, with data pin connected to:
dhtDevice = adafruit_dht.DHT11(18)

camera = PiCamera() # Initialize Pi Camera
camera.resolution = (400, 400) # Set Resolution of Image

def send_Picture():
    # Send Farm image every 4 hours
    try:
        while True:

            import smtplib # Libraru=y to send mail
            from email.mime.multipart import MIMEMultipart 
            from email.mime.text import MIMEText 
            from email.mime.base import MIMEBase 
            from email import encoders
            temperature_c = dhtDevice.temperature# Calculating temp in Celcius
            temperature_f = temperature_c * (9 / 5) + 32# Calculating temp in Farenheit
            humidity = dhtDevice.humidity# Calculating Humidity
            state = GPIO.input(channel)
            if(state==1):
                moisture="Low Moisture Detected!!"
            else:
                moisture="High Moisture Detected!!"
            print(
                "Temp: {:.1f} F / {:.1f} C    Humidity: {}%    {}".format(
                    temperature_f, temperature_c, humidity , moisture
                )
            )
            
            img = str(datetime.now())[:-7]+'.png'
            camera.capture(img) # Capture an image and save
               
            fromaddr = "sudatinikhitha@gmail.com" #User MAil Address
            password = "xsxtsupeapzgcokq"  # User Password

            
            toaddr = "srijavemula001@gmail.com" #Receiver Mail Receiver
            
            
            message = MIMEMultipart() # instance of MIMEMultipart 
            message['From'] = fromaddr # storing the senders email address  
            message['To'] = toaddr # storing the receivers email address  
            message['Subject'] = "Image of Farm" # storing the subject 
              
            
            body = "Find the Attached File. \nImage Time: "+img[:-3]+"\nTemperature: "+ str(temperature_c)+"^C /"+str(temperature_f)+"^F"+"\nHumidity: "+str(humidity)+"%"+"\n Moisture Status: "+str(moisture)# string to store the body of the mail 
              
            message.attach(MIMEText(body, 'plain')) # attach the body with the msg instance 
            
            filename = img
            attachment = open(img, "rb") # open the file to be sent  
              
            
            picture = MIMEBase('application', 'octet-stream') # instance of MIMEBase and named as p 
            
            picture.set_payload((attachment).read()) # To change the payload into encoded form
            
            encoders.encode_base64(picture) # encode into base64 
               
            picture.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
              
            message.attach(picture) # attach the instance 'p' to instance 'msg' 
              
            server = smtplib.SMTP('smtp.gmail.com', 587) # creates SMTP session 
              
            server.starttls() # start TLS for security 
              
            server.login(fromaddr, password) # Authentication 
              
            text = message.as_string() # Converts the Multipart msg into a string 
              
            server.sendmail(fromaddr, toaddr, text) # sending the mail 
              
            server.quit() # terminating the session
          

            time.sleep(4 * 3600)
    

    except Exception as e:
        print('ERR :',e)